In [ ]:
#Importación de las librerías usadas en el ETL
import pandas as pd
import numpy as np
import ast
from datetime import date

In [ ]:
#Carga de dataset
df_movies = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\movies_dataset.csv')
df_credits = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\credits.csv')

C:\Users\house\AppData\Local\Temp\ipykernel_57248\1599109543.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\movies_dataset.csv')


In [ ]:
#Exploración de columnas he información de los datasets
print(df_credits.dtypes)
print(df_movies.dtypes)

In [ ]:
df_movies.head(2)

In [66]:
#Se elimina las columnas que no son de utilidad para el proyecto
del_cols = ['adult', 'belongs_to_collection', 'homepage', 'imdb_id', 'original_title', 'overview', 'poster_path',
'production_companies', 'production_countries', 'runtime', 'spoken_languages', 'tagline', 'video']
df_movies.drop(labels=del_cols, axis=1, inplace=True)

In [67]:
#Se normaliza el tipo de las columnas id para hacer un join y unir ambas tablas
df_credits['id'] = df_credits['id'].astype(str)
df_inner_movies_credits = pd.merge(df_movies, df_credits, on='id', how='inner')
df_inner_movies_credits.head(2)

,budget,genres,id,original_language,popularity,release_date,revenue,status,title,vote_average,vote_count,cast,crew
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [ ]:
#Se verifica si existen valores nulos en el data set
print(df_inner_movies_credits.isnull().sum())

budget                0
genres                0
id                    0
original_language    11
popularity            3
release_date         87
revenue               3
status               84
title                 3
vote_average          3
vote_count            3
cast                  0
crew                  0
dtype: int64


In [71]:
df_inner_movies_credits.fillna({'revenue':0}, inplace=True) #se reemplazan los valores nulos encontrados por 0

#Se transforma la columna 'budget' y 'popularity' a formato numérico
df_inner_movies_credits['budget'] = pd.to_numeric(df_inner_movies_credits['budget'], downcast='signed', errors='coerce')
df_inner_movies_credits['popularity'] = pd.to_numeric(df_inner_movies_credits['popularity'], downcast='signed', errors='coerce')
df_inner_movies_credits['original_language'] = df_inner_movies_credits['original_language'].astype(str)
df_inner_movies_credits['status'] = df_inner_movies_credits['status'].astype(str)
df_inner_movies_credits['title'] = df_inner_movies_credits['title'].astype(str)
df_inner_movies_credits['vote_average'] = pd.to_numeric(df_inner_movies_credits['vote_average'], downcast='signed', errors='coerce')
df_inner_movies_credits['vote_count'] = pd.to_numeric(df_inner_movies_credits['vote_count'], downcast='signed', errors='coerce')
df_inner_movies_credits['vote_count'] = df_inner_movies_credits['vote_count'].astype('Int64')

In [72]:
#Transformación de columna 'release-date' en una clase datetime y 
df_inner_movies_credits['release_date'] = pd.to_datetime(df_inner_movies_credits['release_date'], errors='coerce')

#Se agrega la columna del año
df_inner_movies_credits['year'] = df_inner_movies_credits['release_date'].dt.year.astype('Int64')

In [73]:
#Se crea la columna 'return'
df_inner_movies_credits['return'] = df_inner_movies_credits['revenue']/df_inner_movies_credits['budget']
df_inner_movies_credits['return'] = df_inner_movies_credits['return'].replace([np.nan, np.inf], 0) #reemplazo con 0 los valores que presentan errores

In [74]:
df_inner_movies_credits.head(2)

,budget,genres,id,original_language,popularity,release_date,revenue,status,title,vote_average,vote_count,cast,crew,year,return
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1995,12.451801
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",1995,4.043035


In [ ]:
#Se hace reducción del dataset eliminando valores nulos y filas con datos que no serán usados para las consultas
#También se filtra el dataset para reducir el número de registros
df_inner_movies_credits.dropna(inplace=True)
drop_language = ['en', 'fr', 'it', 'de', 'ar', 'es','uk']
df_inner_movies_credits.query("status == 'Released' and popularity >= 3.0 and release_date >= '1950-1-1' and original_language == @drop_language", inplace=True)
df_inner_movies_credits.reset_index(drop=True, inplace=True)

In [91]:
#Desanidación de columna 'genre' e incorporación a dataset original
df_generos = pd.DataFrame(columns=['genres'])   #Se crea df auxiliar
for index_1, row_1 in df_inner_movies_credits.iterrows():     #Se itera sobre la columna 'genre' con la información anidada
    x = ast.literal_eval(row_1['genres'])       #Se transforma a un objeto lista
    df_dict = pd.DataFrame.from_dict(x)         #Se crea un df
    
    ls = []
    
    for index_2, row_2 in df_dict.iterrows():   #Se itera sobre el df_dict para extraer todos los datos de la columna 'name' y se agregan a la lista
        ls.append(row_2['name'])
    
    df_generos.loc[index_1, 'genres'] = ls      #Se agrega la información al df auxiliar

#Se reemplazan las columnas en el df original
df_inner_movies_credits.drop(labels='genres', axis=1, inplace=True)
df_inner_movies_credits = pd.concat([df_inner_movies_credits, df_generos], axis=1)

In [ ]:
#Creamos iteraciones por las columnas de 'crew' y 'cast' bajo el mismo concepto de las funciones anteriores
#para extraer todos los registros de nombre de actores, nombres del equipo y trabajo y almacenarlos en listas

ls_name_crew = []  
for index, row in df_inner_movies_credits.iterrows():      
    for index in ast.literal_eval(row['crew']):
        ls_name_crew.append(index['name'])

ls_job_crew = []  
for index, row in df_inner_movies_credits.iterrows():      
    for index in ast.literal_eval(row['crew']):
        ls_job_crew.append(index['job'])

ls_name_cast = []  
for index, row in df_inner_movies_credits.iterrows():      
    for index in ast.literal_eval(row['crew']):
        ls_name_cast.append(index['name'])

In [ ]:
#Se crean los dataframes referentes al cast y al crew
df_cast = pd.DataFrame((zip(ls_name_cast)), columns = ['name'])
df_crew = pd.DataFrame((zip(ls_name_crew, ls_job_crew)), columns = ['name', 'job'])

In [ ]:
#Se hace un join con el fin de cruzar la información entre ambos dataframes y poder conocer los actores que son directores
df_directores = pd.merge(df_cast, df_crew, on='name', how='left')

#Se hace una normalización para limpiar los registros duplicados y que no sirvan
df_directores.drop_duplicates('name', keep='first', inplace=True)
df_directores.reset_index(drop=True, inplace=True)
df_directores.query('job == "Director"', inplace=True)
df_directores.drop(labels='job', axis=1, inplace=True)

In [ ]:
#Funciones para la desanidación de las columnas de 'cast' y 'crew'
def actores(texto):
    lista = []                              #Crea una lista vacía donde agrega los strings iterados
    contador = 0                            #Iniciamos contador para extraer el actor principal
    for i in ast.literal_eval(texto):       #Se comienza la iteración con la función ast.literal_eval() que permite la iteración como un diccionario
        if contador < 1:
            lista.append(i['name'])         #se agrega el valor iterado dentro de la clave 'name' a la lista
        contador += 1
    return lista

def directores(texto):
    lista = []    
    for i in ast.literal_eval(texto):
        if i['job'] == 'Director':
            lista.append(i['name'])
    return lista

In [ ]:
#Se pasa la función anterior por cada elemento de la columna para desanidar la información
df_inner_movies_credits['cast'] = df_inner_movies_credits['cast'].apply(actores)
df_inner_movies_credits['crew'] = df_inner_movies_credits['crew'].apply(directores)
df_inner_movies_credits['cast'] = df_inner_movies_credits['cast'].apply(lambda x: ', '.join(x)) #Transformamos a string la columna de cast para poder usar de manera más fácil al actor
df_inner_movies_credits.head(3)

In [ ]:
#Se guardan los datasets
df_inner_movies_credits.to_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\joined_dataset.csv', index=False)
df_directores.to_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\directores.csv', index=False)

Se crean las funciones

def **cantidad_filmaciones_mes( *`Mes`* )**:
Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.

Cargamos los datasets ya normalizados y damos formato de fecha a la columna 'release_data'

In [ ]:
df = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\joined_dataset.csv')
df_dir = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\directores.csv')
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [ ]:
def cantidad_filmaciones_mes(mes):    
    
    if mes == 'enero':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 1).count())
        x = int(ser.iloc[1])
    if mes == 'febrero':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 2).count())
        x = int(ser.iloc[1])
    if mes == 'marzo':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 3).count())
        x = int(ser.iloc[1])
    if mes == 'abril':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 4).count())
        x = int(ser.iloc[1])
    if mes == 'mayo':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 5).count())
        x = int(ser.iloc[1])
    if mes == 'junio':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 6).count())
        x = int(ser.iloc[1])
    if mes == 'julio':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 7).count())
        x = int(ser.iloc[1])
    if mes == 'agosto':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 8).count())
        x = int(ser.iloc[1])
    if mes == 'septiembre':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 9).count())
        x = int(ser.iloc[1])
    if mes == 'octubre':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 10).count())
        x = int(ser.iloc[1])
    if mes == 'noviembre':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 11).count())
        x = int(ser.iloc[1])
    if mes == 'diciembre':
        ser = pd.Series(df['release_date'].groupby(df.release_date.dt.month == 12).count())
        x = int(ser.iloc[1])
        
    print(f"{x} cantidad de películas fueron estrenadas en el mes de {mes}")

def **cantidad_filmaciones_dia( *`Dia`* )**:
Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.

In [ ]:
def cantidad_filmaciones_dia(dia):
    if dia == 'domingo':
        matriz = pd.Series(df['release_date'].groupby(df.release_date.dt.day_name()).count())
        x = int(matriz.iloc[3])
    if dia == 'lunes':
        matriz = pd.Series(df['release_date'].groupby(df.release_date.dt.day_name()).count())
        x = int(matriz.iloc[1])
    if dia == 'martes':
        matriz = pd.Series(df['release_date'].groupby(df.release_date.dt.day_name()).count())
        x = int(matriz.iloc[5])
    if dia == 'miercoles':
        matriz = pd.Series(df['release_date'].groupby(df.release_date.dt.day_name()).count())
        x = int(matriz.iloc[6])
    if dia == 'jueves':
        matriz = pd.Series(df['release_date'].groupby(df.release_date.dt.day_name()).count())
        x = int(matriz.iloc[4])
    if dia == 'viernes':
        matriz = pd.Series(df['release_date'].groupby(df.release_date.dt.day_name()).count())
        x = int(matriz.iloc[0])
    if dia == 'sabado':
        matriz = pd.Series(df['release_date'].groupby(df.release_date.dt.day_name()).count())
        x = int(matriz.iloc[2])
        
    print(f"{x} cantidad de películas fueron estrenadas en los días {dia}")

def **score_titulo( *`titulo_de_la_filmación`* )**:
Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.

In [ ]:
def score_titulo(titulo_de_la_filmacion):
    row_number = df.index.get_loc(df[df['title'] == titulo_de_la_filmacion].index[0])
    anio = int(df['year'][row_number])
    score = float(df['popularity'][row_number])    

    print(f'La película {titulo_de_la_filmacion} fue estrenada en el año {anio} con un score/popularidad de {score}')

def **votos_titulo( *`titulo_de_la_filmación`* )**:
Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

In [ ]:
def votos_titulo(titulo_de_la_filmacion):
    
    row_number = df.index.get_loc(df[df['title'] == titulo_de_la_filmacion].index[0])
    anio = int(df['year'][row_number])    
    promedio = float(df['vote_average'][row_number])
    val = int(df['vote_count'][row_number])
    if int(df['vote_count'][row_number]) >= 2000:
        print(f'La película {titulo_de_la_filmacion} fue estrenada en el año {anio}. La misma cuenta con un total de {val} valoraciones, con un promedio de {promedio}')
    else:
        print('La película tiene menos de 2000 valoraciones')

def **get_actor( *`nombre_actor`* )**:
Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. **La definición no deberá considerar directores.**

In [ ]:
def get_actor(nombre_actor):
        
    director = df_dir.query('name == @nombre_actor')
    existencia = director['name'].count()

    if existencia > 0:
        print('El actor ha trabajado como director')

    else:
        matriz = df.query('cast == @nombre_actor')  
        peliculas = matriz['id'].count()
        retorno = matriz['return'].sum()
        promedio = matriz['return'].mean()

        print(f'El actor {nombre_actor} ha participado de {peliculas} cantidad de filmaciones, el mismo ha conseguido un retorno de {retorno} con un promedio de {promedio} por filmación')
    

def **get_director( *`nombre_director`* )**:
Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [ ]:
def get_director(nombre_director):
    
    df_ord = df.sort_values(by='return', ascending=False)
    
    ls = ['title', 'realese_date', 'return', 'budget', 'revenue']    
    
    for index, row in df_ord.iterrows():
        if nombre_director not in row['crew']:
            print('El nombre ingresado no es un director')
            return None

    print(ls)
    for index, row in df_ord.iterrows():
        lista = []
        if nombre_director in row['crew']:
                lista.append(df.iloc[index]['title'])
                lista.append(df.iloc[index]['release_date'])
                lista.append(str(df.iloc[index]['return']))
                lista.append(str(df.iloc[index]['budget']))
                lista.append(str(df.iloc[index]['revenue']))
                print(lista)

In [10]:
get_director('Robin Williams')

El nombre ingresado no es un director
